In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
import sys
import pickle
from datetime import datetime


In [2]:

sys.stdout=open('test_results.txt','a')

In [4]:

TrainData = pd.read_csv('/Users/SulMac/Desktop/MiskDSI/FPL/Best-FPL-team-predictor/Data/Train/Train.csv')
TestData = pd.read_csv('/Users/SulMac/Desktop/MiskDSI/FPL/Best-FPL-team-predictor/Data/Test/Test.csv')

In [55]:
Traindata_refined = TrainData.loc[:,[i for i in list(TrainData.columns) if i not in ['name','team_x', 'opp_team_name', 'season_x']]]

Traindata_refined = pd.get_dummies(Traindata_refined, columns = ['position', 'GW'], drop_first=True)

Trcols = Traindata_refined.columns[Traindata_refined.isna().sum()>0]
for c in Trcols:
    Traindata_refined[c].fillna(0,inplace=True)
    

In [23]:
Traindata_refined.columns[Traindata_refined.isna().sum()>0]


Index([], dtype='object')

Index([], dtype='object')

In [56]:
#xTrain = Traindata_refined.loc[:, Traindata_refined.columns != 'total_points']
#yTrain = Traindata_refined['total_points']
#xTest = Testdata_refined.loc[:, Testdata_refined.columns != 'total_points']
X_train, X_test, y_train, y_test = train_test_split(Traindata_refined.loc[:, Traindata_refined.columns != 'total_points'], Traindata_refined['total_points'], test_size=0.2, random_state=123)


In [61]:
results_train = {}
results_test = {}

In [36]:
Traindata_refined.shape , xTest.shape

((73837, 80), (24565, 69))

In [26]:
Testdata_refined.shape

(24565, 70)

In [40]:
TrainData.shape , TestData.shape

((73837, 37), (24565, 37))

In [41]:
TrainData.shape , TestData.shape

((73837, 37), (24565, 37))

In [20]:
TrainData.columns

Index(['Unnamed: 0', 'season_x', 'name', 'position', 'team_x', 'assists',
       'bonus', 'bps', 'clean_sheets', 'creativity', 'element', 'fixture',
       'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'minutes',
       'opponent_team', 'opp_team_name', 'own_goals', 'penalties_missed',
       'penalties_saved', 'red_cards', 'round', 'saves', 'selected',
       'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'GW'],
      dtype='object')

In [21]:
TestData.columns

Index(['Unnamed: 0', 'season_x', 'name', 'position', 'team_x', 'assists',
       'bonus', 'bps', 'clean_sheets', 'creativity', 'element', 'fixture',
       'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'minutes',
       'opponent_team', 'opp_team_name', 'own_goals', 'penalties_missed',
       'penalties_saved', 'red_cards', 'round', 'saves', 'selected',
       'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'GW'],
      dtype='object')

In [58]:
#Random Forest
reg = RandomForestRegressor(random_state=0, criterion = 'mse')
#Apply grid search for best parameters
params = {'randomforestregressor__n_estimators' : range(100,500,200),
          'randomforestregressor__min_samples_split' : range(2,10,3)}
pipe = make_pipeline(reg)
grid = GridSearchCV(pipe, param_grid = params, scoring='neg_mean_squared_error', n_jobs=-1, cv=5)
reg = grid.fit(X_train, y_train)

In [62]:
print('Best MSE for Random Forest Regressor: ', grid.best_score_)
print(f'Best Parameters for Random Forest Regressor: Estimators: {grid.best_estimator_} and Min Samples Split: {grid.n_splits_} Params: {grid.best_params_}')

y_train_pred = reg.predict(X_train)
y_test_pred = reg.predict(X_test)
tr_err = mean_squared_error(y_train_pred, y_train)
ts_err = mean_squared_error(y_test_pred, X_test)
print("Training Error: " + str(tr_err)+ " Testing Error: " + str(ts_err))
results_train['random_forest'] = tr_err
results_test['random_forest'] = ts_err

ValueError: y_true and y_pred have different number of output (1!=79)

In [30]:
 print("*"*30+"Random Forest"+"*"*30)

In [49]:
from sklearn.linear_model import LinearRegression
lRegressor = LinearRegression()
lRegressor.fit(xTrain, yTrain)


LinearRegression()

In [63]:
#Linear Regression
reg = LinearRegression()
#Apply grid search for best parameters
params = {}
pipe = make_pipeline(reg)
grid = GridSearchCV(pipe, param_grid = params, scoring='neg_mean_squared_error', n_jobs=-1,  cv=5)
reg = grid.fit(X_train, y_train)
print('Best MSE: ', grid.best_score_)
print('Best Parameters: ', grid.best_estimator_)

y_train_pred = reg.predict(X_train)
y_test_pred = reg.predict(X_test)
tr_err = mean_squared_error(y_train_pred, y_train)
ts_err = mean_squared_error(y_test_pred, y_test)
print("Training Error: " + str(tr_err)+ " Testing Error: " + str(ts_err))
results_train['Linear_regression'] = tr_err
results_test['Linear_regression'] = ts_err

In [64]:
#GBR
reg = GradientBoostingRegressor()
params = {'gradientboostingregressor__n_estimators' : range(100,300,50),
          'gradientboostingregressor__min_samples_split' : range(2,10,2)}
pipe = make_pipeline(reg)
grid = GridSearchCV(pipe, param_grid = params, scoring='neg_mean_squared_error', n_jobs=-1,cv=5)
reg = grid.fit(X_train, y_train)
print('Best MSE: ', grid.best_score_)
print('Best Parameters: ', grid.best_estimator_)

y_train_pred = reg.predict(X_train)
y_test_pred = reg.predict(X_test)
tr_err = mean_squared_error(y_train_pred, y_train)
ts_err = mean_squared_error(y_test_pred, y_test)
print("Training Error: " + str(tr_err)+ " Testing Error: " + str(ts_err))

In [65]:
#Lasso Linear Regression
reg = linear_model.Lasso()
#Apply grid search for best parameters
params = {'lasso__alpha':[0.001, 0.01, 0.1, 0.5, 0.9]}
pipe = make_pipeline(reg)
grid = GridSearchCV(pipe, param_grid = params, scoring='neg_mean_squared_error', n_jobs=-1, cv=5)
reg = grid.fit(X_train, y_train)
print('Best MSE: ', grid.best_score_)
print('Best Parameters: ', grid.best_estimator_)

y_train_pred = reg.predict(X_train)
y_test_pred = reg.predict(X_test)
tr_err = mean_squared_error(y_train_pred, y_train)
ts_err = mean_squared_error(y_test_pred, y_test)
print("Training Error: " + str(tr_err)+ " Testing Error: " + str(ts_err))
results_train['Lasso_regression'] = tr_err
results_test['Lasso_regression'] = ts_err

/Users/SulMac/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9212.299053436556, tolerance: 34.923868437202415
  model = cd_fast.enet_coordinate_descent(
/Users/SulMac/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9239.4620034015, tolerance: 34.9555967283911
  model = cd_fast.enet_coordinate_descent(
/Users/SulMac/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9259.631990327527, tolerance: 34.66898438260321
  model = cd_fast.enet_coordinate_descent(
/Users/SulMac/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_m

In [66]:
print(datetime.now())

In [ ]:
sys.stdout.close()